### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
import numpy as np


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return np.asarray(
            (l_c*r_s - l_s*r_c)**2 / (l_c * r_c * (l_c + r_c)**2)
        ).sum(axis=0)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return np.asarray(
            -(l_s + r_s) / (l_c + r_c) * np.nan_to_num(
                np.log((l_s + r_s) / (l_c + r_c)), nan=0, copy=False
            ) + l_c / (l_c + r_c) * l_s / l_c * np.nan_to_num(
                np.log(l_s / l_c), nan=0, copy=False
            ) + r_c / (l_c + r_c) * r_s / r_c * np.nan_to_num(
                np.log(r_s / r_c), nan=0, copy=False
            )).sum(axis=0)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return ((1 - (l_s + r_s) / (l_c + r_c)).min(axis=0)
                + l_c / (l_c + r_c) * (1 - l_s / l_c).min(axis=0)
                + r_c / (l_c + r_c) * (1 - r_s / r_c).min(axis=0))[0]

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        # TypeError: 'range' object does not support item assignment
        # видоизменение
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature).astype(int):]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        # TypeError: 'range' object does not support item assignment
        # видоизменение
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature).astype(int):]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # l_s, r_s вектора, где указано количество каждого класса
        # в левом и правом поддеревьях,
        # l_c, r_c это число объектов в левом и правом поддеревьях
        # returns Q, threshold

        order = np.argsort(x)
        x = x[order]
        y = y[order]
        *_, indices = np.unique(x, return_index=True)
        indices = indices[1::] - 1  # possible thresholds indices
        if indices.shape[0] == 0:
            return None, None

        # O(X.shape[0]) complexity
        classes = np.arange(self.num_class)
        lr_s = classes[::, np.newaxis] == y[np.newaxis, ::]
        r_s = np.cumsum(lr_s, axis=1)[::, :-1:]
        l_s = np.cumsum(lr_s[::, ::-1], axis=1)[::, -2::-1]
        l_c = l_s.sum(axis=0)[np.newaxis, ::]
        r_c = r_s.sum(axis=0)[np.newaxis, ::]

        Q_s = self.G_function(l_c, l_s, r_c, r_s)[indices]
        ind = np.argmax(Q_s)
        return Q_s[ind], x[indices[ind]]

    def __fit_node(self, x, y, node_id, depth):
        if node_id == 0:
            self.tree = {}
            self.feature_importances_ = np.zeros((x.shape[1]))

        self.tree[node_id] = np.array([None, None, None])
        node = self.tree[node_id]

        classes = np.arange(self.num_class)
        ratios = (classes[::, np.newaxis] == y[np.newaxis, ::]).sum(axis=1)
        ratios = ratios / ratios.sum()

        min_split_criterion = y.shape[0] < self.min_samples_split
        sufficient_criterion = ratios.max() >= self.sufficient_share
        max_depth_criterion = depth == self.max_depth

        if min_split_criterion or sufficient_criterion or max_depth_criterion:
            # мы в листе
            node[0] = self.__class__.LEAF_TYPE
            node[2] = ratios
            node[1] = np.argmax(node[2])
        else:
            node[0] = self.__class__.NON_LEAF_TYPE
            feature_ids = self.get_feature_ids(x.shape[1])
            Q_s = []
            thresholds = []
            features = []
            for feature_id in feature_ids:
                Q, threshold = self.__find_threshold(x[::, feature_id], y)
                if (Q, threshold) != (None, None):
                    Q_s.append(Q)
                    thresholds.append(threshold)
                    features.append(feature_id)
            assert len(Q_s) != 0, "can't split :("
            ind = np.argmax(Q_s)
            node[1] = features[ind]
            node[2] = thresholds[ind]

            left_mask = x[::, node[1]] > node[2]
            right_mask = np.logical_not(left_mask)

            left = y[left_mask]
            right = y[right_mask]
            l_s = (classes[::, np.newaxis] == left[np.newaxis, ::]).sum(axis=1)
            r_s = (classes[::, np.newaxis] == right[np.newaxis, ::]).sum(axis=1)
            l_c = l_s.sum()
            r_c = r_s.sum()
            importance = np.zeros((x.shape[1]))
            importance[node[1]] = 1.0 * self.__gini(l_c, l_s, r_c, r_s)
            self.feature_importances_ += importance
            #node[3] = x.shape[0] * gini  # feature importance

            self.__fit_node(
                x[left_mask], y[left_mask], 2 * node_id + 1, depth + 1
            )
            self.__fit_node(
                x[right_mask], y[right_mask], 2 * node_id + 2, depth + 1
            )
            
            if node_id == 0:
                self.feature_importances_ /= self.feature_importances_.sum()

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 2.18 ms, sys: 328 µs, total: 2.51 ms
Wall time: 1.78 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 20.3 ms, sys: 69 µs, total: 20.4 ms
Wall time: 21.9 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8307692307692308

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8307692307692308

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv("Speed Dating Data.csv", encoding='latin1')

In [10]:
df = df.iloc[:, :97]

In [11]:
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)

In [12]:
df = df.drop(['condtn'], axis=1)

In [13]:
df = df.drop(['round'], axis=1)

In [14]:
df = df.drop(['position', 'positin1'], axis=1)

In [15]:
df = df.drop(['order'], axis=1)

In [16]:
df = df.drop(['partner'], axis=1)

In [17]:
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)

In [18]:
df = df.dropna(subset=['age'])

In [19]:
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)

In [20]:
df = df.drop(['field'], axis=1)

In [21]:
df = pd.concat([df, pd.get_dummies(df.field_cd, prefix='field_cd')], axis=1).drop(["field_cd"], axis=1)

In [22]:
df = df.drop(['undergra'], axis=1)

In [23]:
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)

In [24]:
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].fillna(0)

In [25]:
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)

In [26]:
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].fillna(df.drop_duplicates('iid').tuition.dropna().mean())

In [27]:
df = pd.concat([df, pd.get_dummies(df.race, prefix='race')], axis=1).drop(["race"], axis=1)
df

,iid,gender,wave,pid,match,int_corr,samerace,age,mn_sat,tuition,...,field_cd_15.0,field_cd_16.0,field_cd_17.0,field_cd_18.0,field_cd_19.0,race_1.0,race_2.0,race_3.0,race_4.0,race_6.0
0,1,0,1,11.0,0,0.14,0,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,1,0
1,1,0,1,12.0,0,0.54,0,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,1,0
2,1,0,1,13.0,1,0.16,1,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,1,0
3,1,0,1,14.0,1,0.61,0,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,1,0
4,1,0,1,15.0,1,0.21,0,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,1,21,526.0,0,0.64,0,25.0,0.0,21009.030043,...,0,0,0,1,0,0,1,0,0,0
8374,552,1,21,527.0,0,0.71,0,25.0,0.0,21009.030043,...,0,0,0,1,0,0,1,0,0,0
8375,552,1,21,528.0,0,-0.46,0,25.0,0.0,21009.030043,...,0,0,0,1,0,0,1,0,0,0
8376,552,1,21,529.0,0,0.62,0,25.0,0.0,21009.030043,...,0,0,0,1,0,0,1,0,0,0


In [28]:
df = df.dropna(subset=['imprelig', 'imprace'])

In [29]:
df = df.drop(['from', 'zipcode'], axis=1)

In [30]:
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

In [31]:
df = df.drop(['income'], axis=1)
#df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

In [32]:
df = df.dropna(subset=['date'])

In [33]:
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)

In [34]:
df = df.drop(['career'], axis=1)

In [35]:
df = pd.concat([df, pd.get_dummies(df.career_c, prefix='career_c')], axis=1).drop(["career_c"], axis=1)
df

,iid,gender,wave,pid,match,int_corr,samerace,age,mn_sat,tuition,...,career_c_9.0,career_c_10.0,career_c_11.0,career_c_12.0,career_c_13.0,career_c_14.0,career_c_15.0,career_c_16.0,career_c_17.0,career_c_18.0
0,1,0,1,11.0,0,0.14,0,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,0,1
1,1,0,1,12.0,0,0.54,0,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,0,1
2,1,0,1,13.0,1,0.16,1,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,0,1
3,1,0,1,14.0,1,0.61,0,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,0,1
4,1,0,1,15.0,1,0.21,0,21.0,0.0,21009.030043,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,1,21,526.0,0,0.64,0,25.0,0.0,21009.030043,...,0,0,0,0,0,0,1,0,0,0
8374,552,1,21,527.0,0,0.71,0,25.0,0.0,21009.030043,...,0,0,0,0,0,0,1,0,0,0
8375,552,1,21,528.0,0,-0.46,0,25.0,0.0,21009.030043,...,0,0,0,0,0,0,1,0,0,0
8376,552,1,21,529.0,0,0.62,0,25.0,0.0,21009.030043,...,0,0,0,0,0,0,1,0,0,0


In [36]:
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)

In [37]:
df = df.drop(['expnum'], axis=1)

In [38]:
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']

In [39]:
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]

In [40]:
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

In [41]:
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)

In [42]:
temp.loc[idx, ]

,iid,wave,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,totalsum
918,67,3,20.0,15.0,20.0,20.0,5.0,10.0,90.0
1530,105,4,30.0,15.0,20.0,20.0,0.0,5.0,90.0
7221,489,19,20.0,10.0,20.0,20.0,20.0,0.0,90.0
7586,517,21,15.0,20.0,20.0,20.0,5.0,10.0,90.0
7784,526,21,10.0,10.0,30.0,20.0,10.0,15.0,95.0


In [43]:
idx = ((temp.wave >= 6) & (temp.wave <= 9))

In [44]:
temp.loc[idx, ]

,iid,wave,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,totalsum
1846,132,6,16.67,16.67,16.67,16.67,16.67,16.67,100.02
1851,133,6,12.77,19.15,17.02,17.02,14.89,19.15,100.00
1856,134,6,6.67,20.00,20.00,17.78,20.00,15.56,100.01
1861,135,6,18.18,22.73,18.18,13.64,13.64,13.64,100.01
1871,137,6,17.39,17.39,15.22,17.39,13.04,19.57,100.00
...,...,...,...,...,...,...,...,...,...
3308,229,9,21.43,16.67,21.43,16.67,11.90,11.90,100.00
3328,230,9,23.81,23.81,23.81,23.81,2.38,2.38,100.00
3348,231,9,18.60,20.93,23.26,23.26,2.33,11.63,100.01
3368,232,9,17.78,17.78,17.78,17.78,13.33,15.56,100.01


In [45]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)

In [46]:
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

In [47]:
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']

In [48]:
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]

In [49]:
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

In [50]:
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)

In [51]:
temp.loc[idx, ]

,iid,wave,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,totalsum
4816,320,12,20.0,10.0,10.0,10.0,20.0,10.0,80.0


In [52]:
idx = ((temp.wave >= 6) & (temp.wave <= 9))

In [53]:
temp.loc[idx, ]

,iid,wave,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,totalsum
1846,132,6,22.73,15.91,13.64,15.91,13.64,18.18,100.01
1851,133,6,24.32,16.22,13.51,16.22,10.81,18.92,100.00
1856,134,6,20.51,15.38,12.82,20.51,12.82,17.95,99.99
1861,135,6,18.75,16.67,14.58,18.75,18.75,12.50,100.00
1871,137,6,14.89,17.02,17.02,17.02,14.89,19.15,99.99
...,...,...,...,...,...,...,...,...,...
3308,229,9,15.22,19.57,15.22,17.39,21.74,10.87,100.01
3328,230,9,16.67,16.67,16.67,16.67,16.67,16.67,100.02
3348,231,9,18.18,11.36,15.91,18.18,20.45,15.91,99.99
3368,232,9,19.15,17.02,17.02,17.02,14.89,14.89,99.99


In [54]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)

In [55]:
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

In [56]:
df = df.drop(['temp_totalsum'], axis=1)

In [57]:
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)

In [58]:
df = df.drop(['wave'], axis=1)

In [59]:
df

,iid,gender,pid,match,int_corr,samerace,age,mn_sat,tuition,imprace,...,career_c_9.0,career_c_10.0,career_c_11.0,career_c_12.0,career_c_13.0,career_c_14.0,career_c_15.0,career_c_16.0,career_c_17.0,career_c_18.0
0,1,0,11.0,0,0.14,0,21.0,0.0,21009.030043,2.0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,12.0,0,0.54,0,21.0,0.0,21009.030043,2.0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,13.0,1,0.16,1,21.0,0.0,21009.030043,2.0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,14.0,1,0.61,0,21.0,0.0,21009.030043,2.0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,15.0,1,0.21,0,21.0,0.0,21009.030043,2.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,1,526.0,0,0.64,0,25.0,0.0,21009.030043,1.0,...,0,0,0,0,0,0,1,0,0,0
8374,552,1,527.0,0,0.71,0,25.0,0.0,21009.030043,1.0,...,0,0,0,0,0,0,1,0,0,0
8375,552,1,528.0,0,-0.46,0,25.0,0.0,21009.030043,1.0,...,0,0,0,0,0,0,1,0,0,0
8376,552,1,529.0,0,0.62,0,25.0,0.0,21009.030043,1.0,...,0,0,0,0,0,0,1,0,0,0


In [60]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'

In [61]:
df_male

,iid,pid,match,int_corr,samerace,age,mn_sat,tuition,imprace,imprelig,...,career_c_9.0,career_c_10.0,career_c_11.0,career_c_12.0,career_c_13.0,career_c_14.0,career_c_15.0,career_c_16.0,career_c_17.0,career_c_18.0
100,11,1.0,0,0.14,0,27.0,0.0,21009.030043,7.0,3.0,...,0,0,0,0,0,0,0,0,0,0
101,11,2.0,0,0.29,1,27.0,0.0,21009.030043,7.0,3.0,...,0,0,0,0,0,0,0,0,0,0
102,11,3.0,0,-0.24,1,27.0,0.0,21009.030043,7.0,3.0,...,0,0,0,0,0,0,0,0,0,0
103,11,4.0,0,-0.18,1,27.0,0.0,21009.030043,7.0,3.0,...,0,0,0,0,0,0,0,0,0,0
104,11,5.0,0,-0.43,1,27.0,0.0,21009.030043,7.0,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,526.0,0,0.64,0,25.0,0.0,21009.030043,1.0,1.0,...,0,0,0,0,0,0,1,0,0,0
8374,552,527.0,0,0.71,0,25.0,0.0,21009.030043,1.0,1.0,...,0,0,0,0,0,0,1,0,0,0
8375,552,528.0,0,-0.46,0,25.0,0.0,21009.030043,1.0,1.0,...,0,0,0,0,0,0,1,0,0,0
8376,552,529.0,0,0.62,0,25.0,0.0,21009.030043,1.0,1.0,...,0,0,0,0,0,0,1,0,0,0


In [62]:
df_female

,iid_f,pid_f,age_f,mn_sat_f,tuition_f,imprace_f,imprelig_f,goal_f,date_f,go_out_f,...,career_c_9.0_f,career_c_10.0_f,career_c_11.0_f,career_c_12.0_f,career_c_13.0_f,career_c_14.0_f,career_c_15.0_f,career_c_16.0_f,career_c_17.0_f,career_c_18.0_f
0,1,11.0,21.0,0.0,21009.030043,2.0,4.0,2.0,7.0,1.0,...,0,0,0,0,0,0,0,0,0,1
10,2,11.0,24.0,0.0,21009.030043,2.0,5.0,1.0,5.0,1.0,...,0,0,0,0,0,0,0,0,0,1
20,3,11.0,25.0,0.0,21009.030043,8.0,4.0,6.0,3.0,1.0,...,0,0,0,0,0,0,0,0,0,1
30,4,11.0,23.0,0.0,21009.030043,1.0,1.0,1.0,5.0,1.0,...,0,0,0,0,0,0,0,0,0,0
40,5,11.0,21.0,0.0,21009.030043,8.0,1.0,2.0,4.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7762,525,531.0,24.0,1360.0,26062.000000,2.0,8.0,2.0,4.0,1.0,...,0,0,0,0,0,0,0,0,0,0
7784,526,531.0,26.0,1188.0,11311.000000,5.0,2.0,1.0,5.0,3.0,...,0,0,1,0,0,0,0,0,0,0
7806,527,531.0,24.0,1290.0,15309.000000,5.0,5.0,1.0,4.0,3.0,...,0,0,1,0,0,0,0,0,0,0
7850,529,531.0,22.0,1430.0,26908.000000,4.0,1.0,2.0,4.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
df_female.pid_f = df_female.pid_f.astype(int)

In [64]:
with open("male_describe.csv", "w") as file:
    file.write(df_male.describe().to_csv())
with open("female_describe.csv", "w") as file:
    file.write(df_female.describe().to_csv())
# это я проверяю, что все пошло хорошо с таблицами и там нет чего-то не того

In [65]:
dataset = pd.merge(df_male, df_female.rename(columns={"iid_f": "pid"}), on="pid", left_index=True)
dataset

,iid,pid,match,int_corr,samerace,age,mn_sat,tuition,imprace,imprelig,...,career_c_9.0_f,career_c_10.0_f,career_c_11.0_f,career_c_12.0_f,career_c_13.0_f,career_c_14.0_f,career_c_15.0_f,career_c_16.0_f,career_c_17.0_f,career_c_18.0_f
0,11,1.0,0,0.14,0,27.0,0.0,21009.030043,7.0,3.0,...,0,0,0,0,0,0,0,0,0,1
0,12,1.0,0,0.54,0,22.0,0.0,21009.030043,1.0,1.0,...,0,0,0,0,0,0,0,0,0,1
0,13,1.0,1,0.16,1,22.0,0.0,21009.030043,3.0,5.0,...,0,0,0,0,0,0,0,0,0,1
0,14,1.0,1,0.61,0,23.0,0.0,21009.030043,1.0,1.0,...,0,0,0,0,0,0,0,0,0,1
0,15,1.0,1,0.21,0,24.0,0.0,21009.030043,3.0,1.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7872,548,530.0,0,0.23,0,30.0,0.0,21009.030043,8.0,8.0,...,0,1,0,0,0,0,0,0,0,0
7872,549,530.0,0,0.29,0,28.0,0.0,21009.030043,2.0,3.0,...,0,1,0,0,0,0,0,0,0,0
7872,550,530.0,0,-0.20,0,30.0,0.0,21009.030043,5.0,6.0,...,0,1,0,0,0,0,0,0,0,0
7872,551,530.0,0,-0.32,0,27.0,1400.0,26019.000000,2.0,1.0,...,0,1,0,0,0,0,0,0,0,0


In [66]:
with open("dataset.csv", "w") as file:
    file.write(dataset.to_csv())

In [67]:
y = np.array(dataset.match)
X = np.array(dataset.drop(["match"], axis=1))[::, ::]

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [69]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [70]:
%time clf.fit(X_train, y_train)

CPU times: user 88 ms, sys: 24 µs, total: 88 ms
Wall time: 86.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [71]:
%time my_clf.fit(X_train, y_train)

CPU times: user 3.76 s, sys: 4.41 ms, total: 3.76 s
Wall time: 3.76 s


## Проверка качества работы на Speed Dating Data

In [72]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5777054604425387

In [73]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.6038267721677424

## Задание 3

Я честно не понимаю, что тут должно быть... Написано сделать под заданием 4 задание 3. Я сделаю. Ну, или попытаюсь...

## Задание 4

In [74]:
np.argsort(clf.feature_importances_)[:10:]

array([ 70, 139,  69,  68,  67,  66,  65,  64,  63,  62])

In [75]:
np.argsort(my_clf.feature_importances_)[:10:]

array([ 70,  71, 139,  69,  68,  67,  66,  65,  64,  63])

## Задание 5

In [76]:
from scipy.stats import randint as randint

In [77]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 10),
    'max_features': ["sqrt", "log2", None],
    'class_weight': [None, 'balanced']
}

In [78]:
cv = KFold(n_splits=5, random_state=123, shuffle=True)

In [79]:
model = RandomForestClassifier(random_state=123, n_estimators=50)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=200, n_jobs=-1,
                                   cv=cv, scoring='f1', random_state=123)
random_search.fit(X, y)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=123, shuffle=True),
                   error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_l...
                                                    warm_start=False),
   

In [80]:
random_search.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'log2'}

In [81]:
random_search.best_score_

0.3694774291159711